<a href="https://colab.research.google.com/github/ElenaNtm/convert-to-DIMAC/blob/main/hedno_stat_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [ ]:
import pandas as pd 
import numpy as np
import math
import operator 
from datetime import datetime
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
from sklearn.ensemble import HistGradientBoostingClassifier

In [ ]:
from sklearn.model_selection import train_test_split
import sklearn

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

!! Αν δεις και γεμιζει η RAΜ κανε επανεκκινηση χρονου εκτελεσης

# The Dataset

## Powerthefts - Target Variable

In [ ]:
path = "/content/drive/MyDrive/HEDNO/PowerThefts_TRAIN.csv"
df_power_thefts = pd.read_csv(path, sep= ',')
df_power_thefts.head()

,Unnamed: 0.1,Unnamed: 0,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY
0,0,0,000A31FE38C6F648845941B642E287FE02000000004D63...,3,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2018-03-01,2018-03-01,NaN,1283.0
1,1,1,000A31FE38C6F648845941B642E287FE0200000000801E...,8,Μέσω Καταμέτρησης (ΔΑΚ),ΑΚΡΟΔΕΚΤΕΣ,2018-08-16,2018-11-01,2018-11-01,640.0
2,2,2,000A31FE38C6F648845941B642E287FE0200000000B03B...,2,Μέσω Λοιπών Εργασιών & Συντήρησης,ΜΗΧΑΝΙΣΜΟΣ ΜΕΤΡΗΣΗΣ,2018-12-05,2018-12-18,NaN,4665.0
3,3,3,000A31FE38C6F648845941B642E287FE0200000000BCD9...,2,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΚΡΟΔΕΚΤΕΣ,2017-01-19,2017-01-30,NaN,NaN
4,4,4,000A31FE38C6F648845941B642E287FE02000000017AA3...,5,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-05-29,2019-05-29,NaN,1049.0


In [ ]:
df_power_thefts.drop(['Unnamed: 0.1','Unnamed: 0'], axis=1, inplace = True)

###Check Duplicate lines - No duplicate lines

In [ ]:
#bool_series = df_power_thefts.duplicated()

In [ ]:
#[i for i, x in enumerate(bool_series) if x]

## Rest Train + Test

###Records

In [ ]:
path = "/content/drive/MyDrive/HEDNO/Records_TRAIN.csv"
df_records_train = pd.read_csv(path, sep= '|')

In [ ]:
path = "/content/drive/MyDrive/HEDNO/Records_TEST.csv"
df_records_test = pd.read_csv(path, sep= '|')

###Representations

In [ ]:
path = "/content/drive/MyDrive/HEDNO/Representations_TRAIN.csv"
df_representations_train = pd.read_csv(path, sep= '|')

In [ ]:
path = "/content/drive/MyDrive/HEDNO/Representations_TEST.csv"
df_representations_test = pd.read_csv(path, sep= '|')

###Requests

In [ ]:
path = "/content/drive/MyDrive/HEDNO/Requests_TRAIN.csv"
df_requests_train = pd.read_csv(path, sep= '|')

In [ ]:
path = "/content/drive/MyDrive/HEDNO/Requests_TEST.csv"
df_requests_test = pd.read_csv(path, sep= '|')

###Consumptions

In [ ]:
path = "/content/drive/MyDrive/HEDNO/Consumptions_TRAIN.csv"
df_consumptions_train = pd.read_csv(path, sep= '|')

In [ ]:
path = "/content/drive/MyDrive/HEDNO/Consumptions_TEST.csv"
df_consumptions_test = pd.read_csv(path, sep= '|')

## Data Heads

In [ ]:
df_records_train.head()

,ACCT_NBR,SUCCESSOR,VOLTAGE,PARNO,XRHSH,CONTRACT_CAPACITY,ACCT_CONTROL,ACCT_WGS84_X,ACCT_WGS84_Y
0,000A31FE38C6F648845941B642E287FE0200000000004A...,1,LOW,0,2,12.0,0,"53,422551543686701","19,592757873078753"
1,000A31FE38C6F648845941B642E287FE0200000000008D...,1,LOW,0,2,12.0,0,"53,407108521588633","19,569743639932696"
2,000A31FE38C6F648845941B642E287FE0200000000008D...,3,LOW,2,2,25.0,0,"53,407108521588633","19,569743639932696"
3,000A31FE38C6F648845941B642E287FE0200000000008D...,4,LOW,2,2,25.0,0,"53,407108521588633","19,569743639932696"
4,000A31FE38C6F648845941B642E287FE0200000000008D...,5,LOW,2,2,25.0,0,"53,407108521588633","19,569743639932696"


In [ ]:
df_records_test.head()

,ACCT_NBR,SUCCESSOR,VOLTAGE,PARNO,XRHSH,CONTRACT_CAPACITY,ACCT_CONTROL,ACCT_WGS84_X,ACCT_WGS84_Y
0,000A31FE38C6F648845941B642E287FE0200000000006D...,1,LOW,0,1,12,0,"53,451030471299404","19,600792390085896"
1,000A31FE38C6F648845941B642E287FE0200000000006E...,1,LOW,0,1,12,0,"53,423690495301905","19,615367380585742"
2,000A31FE38C6F648845941B642E287FE020000000001BF...,2,LOW,2,1,25,0,"53,466063546765042","19,606420323080993"
3,000A31FE38C6F648845941B642E287FE02000000000357...,1,LOW,0,1,12,0,"53,466930013537464","19,594032532171479"
4,000A31FE38C6F648845941B642E287FE02000000000536...,1,LOW,0,1,12,0,"53,46529071739247","19,608223560803914"


In [ ]:
df_representations_train.head()

,ACCT_NBR,SUCCESSOR,SUPPLIER,SUPPLIER_TO,START_DATE,END_DATE
0,000A31FE38C6F648845941B642E287FE0200000043989C...,4,W,NaN,2017-04-04,2022-07-19
1,000A31FE38C6F648845941B642E287FE020000004398BA...,1,Δ,NaN,1995-10-18,1999-12-22
2,000A31FE38C6F648845941B642E287FE020000004398BA...,4,Δ,Δ,2019-11-06,2020-06-23
3,000A31FE38C6F648845941B642E287FE020000004398BA...,5,Δ,NaN,2020-06-23,2022-09-16
4,000A31FE38C6F648845941B642E287FE020000004398BD...,1,Δ,Δ,1995-10-15,2000-01-13


In [ ]:
df_representations_test.head()

,ACCT_NBR,SUCCESSOR,SUPPLIER,SUPPLIER_TO,START_DATE,END_DATE
0,000A31FE38C6F648845941B642E287FE02000000DDCF2A...,2,Δ,NaN,1996-11-05,2022-09-16
1,000A31FE38C6F648845941B642E287FE02000000DDCFC0...,1,Δ,L,1998-03-23,2018-02-27
2,000A31FE38C6F648845941B642E287FE02000000DDCFC0...,2,L,W,2018-02-27,2019-10-02
3,000A31FE38C6F648845941B642E287FE02000000DDCFC0...,5,Φ,NaN,2021-12-17,2022-09-16
4,000A31FE38C6F648845941B642E287FE02000000DDD043...,1,Δ,NaN,1996-05-20,2016-05-20


In [ ]:
df_requests_train.head()

,ACCT_NBR,SUCCESSOR,REQUEST_TYPE,REQUEST_DATE,COMPL_REQUEST_STATUS,COMPLETION_DATE
0,000A31FE38C6F648845941B642E287FE020000000AA1CA...,7,recon,2019-02-12,compl,2019-02-13
1,000A31FE38C6F648845941B642E287FE020000000AA1D1...,3,NaN,2015-05-26,compl,2015-05-28
2,000A31FE38C6F648845941B642E287FE020000000AA1D1...,4,NaN,2017-02-06,NaN,2017-03-14
3,000A31FE38C6F648845941B642E287FE020000000AA1D1...,5,discon,2018-06-28,compl,2018-09-27
4,000A31FE38C6F648845941B642E287FE020000000AA1D1...,5,reprChange,2018-09-27,compl,2018-10-02


In [ ]:
df_requests_test.head()

,ACCT_NBR,SUCCESSOR,REQUEST_TYPE,REQUEST_DATE,COMPL_REQUEST_STATUS,COMPLETION_DATE
0,000A31FE38C6F648845941B642E287FE020000000001BF...,1,discon,2014-10-11,compl,2014-11-13
1,000A31FE38C6F648845941B642E287FE020000000001BF...,1,recon,2014-11-14,compl,2014-11-19
2,000A31FE38C6F648845941B642E287FE020000000001BF...,1,recon,2014-11-24,NaN,2014-11-24
3,000A31FE38C6F648845941B642E287FE020000000001BF...,1,recon,2018-02-02,compl,2018-02-02
4,000A31FE38C6F648845941B642E287FE020000000001BF...,1,recon,2018-08-01,compl,2018-08-01


In [ ]:
df_consumptions_train.head()

,ACCT_NBR,SUCCESSOR,BS_RATE,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
0,000A31FE38C6F648845941B642E287FE02000000B7602D...,1,13,000A31FE38C6F648845941B642E287FE0200000032011D...,56,2019-08-26
1,000A31FE38C6F648845941B642E287FE020000003CC307...,1,11,000A31FE38C6F648845941B642E287FE020000002BFDE8...,2236,2022-03-10
2,000A31FE38C6F648845941B642E287FE02000000543AF1...,1,13,000A31FE38C6F648845941B642E287FE0200000055887E...,678,2021-04-20
3,000A31FE38C6F648845941B642E287FE02000000617012...,1,13,000A31FE38C6F648845941B642E287FE02000000310114...,156,2022-04-27
4,000A31FE38C6F648845941B642E287FE020000006CBF0C...,1,13,000A31FE38C6F648845941B642E287FE02000000B94B46...,502,2022-06-03


In [ ]:
df_consumptions_test.head()

,ACCT_NBR,SUCCESSOR,BS_RATE,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
0,000A31FE38C6F648845941B642E287FE0200000011084B...,2,10,000A31FE38C6F648845941B642E287FE020000003C4E97...,412,2022-07-18
1,000A31FE38C6F648845941B642E287FE0200000011084B...,2,10,000A31FE38C6F648845941B642E287FE020000003C4E97...,807,2020-11-17
2,000A31FE38C6F648845941B642E287FE0200000011084B...,1,10,000A31FE38C6F648845941B642E287FE020000003C4E97...,1188,2019-03-14
3,000A31FE38C6F648845941B642E287FE02000000BCD832...,4,10,000A31FE38C6F648845941B642E287FE02000000C30DC8...,1333,2022-09-14
4,000A31FE38C6F648845941B642E287FE02000000BCD832...,2,10,000A31FE38C6F648845941B642E287FE02000000C30DC8...,1110,2019-09-12


##Data Info

In [ ]:
df_records_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1234509 entries, 0 to 1234508
Data columns (total 9 columns):
 #   Column             Non-Null Count    Dtype  
---  ------             --------------    -----  
 0   ACCT_NBR           1234509 non-null  object 
 1   SUCCESSOR          1234509 non-null  int64  
 2   VOLTAGE            1234509 non-null  object 
 3   PARNO              1234509 non-null  int64  
 4   XRHSH              1234509 non-null  int64  
 5   CONTRACT_CAPACITY  1233327 non-null  float64
 6   ACCT_CONTROL       1234509 non-null  int64  
 7   ACCT_WGS84_X       1234509 non-null  object 
 8   ACCT_WGS84_Y       1234509 non-null  object 
dtypes: float64(1), int64(4), object(4)
memory usage: 84.8+ MB


In [ ]:
df_records_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137534 entries, 0 to 137533
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ACCT_NBR           137534 non-null  object
 1   SUCCESSOR          137534 non-null  int64 
 2   VOLTAGE            137534 non-null  object
 3   PARNO              137534 non-null  int64 
 4   XRHSH              137534 non-null  int64 
 5   CONTRACT_CAPACITY  137534 non-null  int64 
 6   ACCT_CONTROL       137534 non-null  int64 
 7   ACCT_WGS84_X       137534 non-null  object
 8   ACCT_WGS84_Y       137534 non-null  object
dtypes: int64(5), object(4)
memory usage: 9.4+ MB


In [ ]:
df_representations_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1232180 entries, 0 to 1232179
Data columns (total 6 columns):
 #   Column       Non-Null Count    Dtype 
---  ------       --------------    ----- 
 0   ACCT_NBR     1232180 non-null  object
 1   SUCCESSOR    1232180 non-null  int64 
 2   SUPPLIER     1232180 non-null  object
 3   SUPPLIER_TO  496905 non-null   object
 4   START_DATE   1232180 non-null  object
 5   END_DATE     1232180 non-null  object
dtypes: int64(1), object(5)
memory usage: 56.4+ MB


In [ ]:
df_representations_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136960 entries, 0 to 136959
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   ACCT_NBR     136960 non-null  object
 1   SUCCESSOR    136960 non-null  int64 
 2   SUPPLIER     136960 non-null  object
 3   SUPPLIER_TO  55809 non-null   object
 4   START_DATE   136960 non-null  object
 5   END_DATE     136960 non-null  object
dtypes: int64(1), object(5)
memory usage: 6.3+ MB


In [ ]:
df_requests_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2346360 entries, 0 to 2346359
Data columns (total 6 columns):
 #   Column                Dtype 
---  ------                ----- 
 0   ACCT_NBR              object
 1   SUCCESSOR             int64 
 2   REQUEST_TYPE          object
 3   REQUEST_DATE          object
 4   COMPL_REQUEST_STATUS  object
 5   COMPLETION_DATE       object
dtypes: int64(1), object(5)
memory usage: 107.4+ MB


In [ ]:
df_requests_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94206 entries, 0 to 94205
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ACCT_NBR              94206 non-null  object
 1   SUCCESSOR             94206 non-null  int64 
 2   REQUEST_TYPE          77332 non-null  object
 3   REQUEST_DATE          94206 non-null  object
 4   COMPL_REQUEST_STATUS  72407 non-null  object
 5   COMPLETION_DATE       93724 non-null  object
dtypes: int64(1), object(5)
memory usage: 4.3+ MB


In [ ]:
df_consumptions_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9201394 entries, 0 to 9201393
Data columns (total 6 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   ACCT_NBR          object
 1   SUCCESSOR         int64 
 2   BS_RATE           object
 3   MS_METER_NBR      object
 4   CSS_MS_HS_USE     int64 
 5   MEASUREMENT_DATE  object
dtypes: int64(2), object(4)
memory usage: 421.2+ MB


In [ ]:
df_consumptions_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 718168 entries, 0 to 718167
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   ACCT_NBR          718168 non-null  object
 1   SUCCESSOR         718168 non-null  int64 
 2   BS_RATE           718168 non-null  int64 
 3   MS_METER_NBR      718168 non-null  object
 4   CSS_MS_HS_USE     718168 non-null  int64 
 5   MEASUREMENT_DATE  718168 non-null  object
dtypes: int64(3), object(3)
memory usage: 32.9+ MB


To consumptions Test BS_RATE είναι integer ενω στο train είναι object και εχει χαρακτηρες μεσα (πχ Ν4)

# Stat Analysis

## Missing values for each dataframe

Records

In [ ]:
total = df_records_train.isnull().sum().sort_values(ascending=False)
percent = (df_records_train.isnull().sum()/df_records_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
CONTRACT_CAPACITY,1182,0.000957
ACCT_NBR,0,0.000000
SUCCESSOR,0,0.000000
VOLTAGE,0,0.000000
PARNO,0,0.000000
XRHSH,0,0.000000
ACCT_CONTROL,0,0.000000
ACCT_WGS84_X,0,0.000000
ACCT_WGS84_Y,0,0.000000


Representations

In [ ]:
total = df_representations_train.isnull().sum().sort_values(ascending=False)
percent = (df_representations_train.isnull().sum()/df_representations_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
SUPPLIER_TO,735275,0.596727
ACCT_NBR,0,0.000000
SUCCESSOR,0,0.000000
SUPPLIER,0,0.000000
START_DATE,0,0.000000
END_DATE,0,0.000000


Κανω drop το SUPPLIER_TO αφου σχεδον το 60% των τιμων ειναι κενες

In [ ]:
df_representations_train.drop(['SUPPLIER_TO'], axis = 1, inplace = True)
df_representations_train.head()

,ACCT_NBR,SUCCESSOR,SUPPLIER,START_DATE,END_DATE
0,000A31FE38C6F648845941B642E287FE0200000043989C...,4,W,2017-04-04,2022-07-19
1,000A31FE38C6F648845941B642E287FE020000004398BA...,1,Δ,1995-10-18,1999-12-22
2,000A31FE38C6F648845941B642E287FE020000004398BA...,4,Δ,2019-11-06,2020-06-23
3,000A31FE38C6F648845941B642E287FE020000004398BA...,5,Δ,2020-06-23,2022-09-16
4,000A31FE38C6F648845941B642E287FE020000004398BD...,1,Δ,1995-10-15,2000-01-13


In [ ]:
#same for test
df_representations_test.drop(['SUPPLIER_TO'], axis = 1, inplace = True)

Requests

In [ ]:
total = df_requests_train.isnull().sum().sort_values(ascending=False)
percent = (df_requests_train.isnull().sum()/df_requests_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
REQUEST_TYPE,433016,0.184548
COMPL_REQUEST_STATUS,423768,0.180607
COMPLETION_DATE,10273,0.004378
ACCT_NBR,0,0.000000
SUCCESSOR,0,0.000000
REQUEST_DATE,0,0.000000


Consumptions

In [ ]:
total = df_consumptions_train.isnull().sum().sort_values(ascending=False)
percent = (df_consumptions_train.isnull().sum()/df_consumptions_train.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
ACCT_NBR,0,0.0
SUCCESSOR,0,0.0
BS_RATE,0,0.0
MS_METER_NBR,0,0.0
CSS_MS_HS_USE,0,0.0
MEASUREMENT_DATE,0,0.0


## Plotting the variables - Not to run

### Records

In [ ]:
#sns.histplot(data=df_records_train, x='VOLTAGE', bins=30)

In [ ]:
#df_records_train['VOLTAGE'].describe()

Επισης το Voltage παει για πεταμα μιας και σχεδον ολα ειναι Low \\
θα τσεκαρω τους χρηστες του med ως προς τις ρευματοκλοπες μηπως υπαρχει ισχυρη συσχετιση (που δεν παιζει) αλλιως φευγει

In [ ]:
#sns.histplot(data=df_records_train, x='PARNO', bins=30)

In [ ]:
#df_records_train['PARNO'].value_counts()

In [ ]:
#sns.histplot(data=df_records_train, x='XRHSH', bins=30)

In [ ]:
#df_records_train['XRHSH'].value_counts()

In [ ]:
#sns.displot(df_records_train['CONTRACT_CAPACITY'],kde=True,color='blue',height=5,aspect=1) 

In [ ]:
#df_records_train['CONTRACT_CAPACITY'].describe()

In [ ]:
#sns.histplot(data=df_records_train, x='ACCT_CONTROL', bins=30)

In [ ]:
#df_records_train['ACCT_CONTROL'].value_counts()

### Representations

In [ ]:
#sns.histplot(data=df_representations_train, x='SUPPLIER')

In [ ]:
#df_representations_train['SUPPLIER'].value_counts()

Ποιοι στην ευχη ειναι ολοι αυτοι οι παροχοι

In [ ]:
#supplier_time = df_representations_train['START_DATE'] - df_representations_train['END_DATE']

θελουν φτιαξιμο οι ημερομηνιες στο representations γτ ειναι αποθηκευμενες ως str. \\
και αντι να εχουμε 2 στηλες start & end μπορουμε απλα να εχουμε τον χρονο παραμονης στον supplier βρισκοντας την διαφορα των 2 \\
βασικα δεν ξερω αν θα μας χρησιμευσει ο χρονος παραμονης \\
αστο γαματο θα το δω αργοτερα

### Requests

In [ ]:
#sns.histplot(data=df_requests_train, x='REQUEST_TYPE')

In [ ]:
#sns.histplot(data=df_requests_train, x='COMPL_REQUEST_STATUS')

In [ ]:
#df_requests_train['COMPL_REQUEST_STATUS'].value_counts()

επισης η συντριπτικη πλειοψηφια ειναι complete

In [ ]:
# na doume ti tha kanoume me ta dates

### Consumption

In [ ]:
#sns.histplot(data=df_consumptions_train, x='BS_RATE')

In [ ]:
#lst=[]
#lst.append(df_consumptions_train['BS_RATE'].unique())

In [ ]:
#lst1=[]
#lst1.append(df_consumptions_test['BS_RATE'].unique())

In [ ]:
#all(x in lst1 for x in lst)

In [ ]:
 #sns.displot(df_consumptions_train['CSS_MS_HS_USE'],kde=True,color='blue',height=5,aspect=1) #αργει παρα πολυ να τρεξει!!!

In [ ]:
#df_consumptions_train['CSS_MS_HS_USE'].describe()

# Adding the variables to our train set

# Creating our df

##Θα χρησιμποιήσουμε τον πίνακα records ως την βάση καθώς έχει βασικά στοιχεία για κάθε χρήστη (λογικά μοναδικές τιμές θα το τσεκάρω)

In [ ]:
df_records_train.drop(['ACCT_WGS84_X', 'ACCT_WGS84_Y'], axis = 1, inplace = True)
df_records_train.head()

,ACCT_NBR,SUCCESSOR,VOLTAGE,PARNO,XRHSH,CONTRACT_CAPACITY,ACCT_CONTROL
0,000A31FE38C6F648845941B642E287FE0200000000004A...,1,LOW,0,2,12.0,0
1,000A31FE38C6F648845941B642E287FE0200000000008D...,1,LOW,0,2,12.0,0
2,000A31FE38C6F648845941B642E287FE0200000000008D...,3,LOW,2,2,25.0,0
3,000A31FE38C6F648845941B642E287FE0200000000008D...,4,LOW,2,2,25.0,0
4,000A31FE38C6F648845941B642E287FE0200000000008D...,5,LOW,2,2,25.0,0


In [ ]:
#same for test
df_records_test.drop(['ACCT_WGS84_X', 'ACCT_WGS84_Y'], axis = 1, inplace = True)

Είναι όντως μοναδικές

In [ ]:
#bool_series = df_records_train.duplicated()
#[i for i, x in enumerate(bool_series) if x]

##drop στήλες που δεν θα χρησιμοποιήσουμε

In [ ]:
df_representations_train.drop(['END_DATE'], axis = 1, inplace = True)# έχω κάνει drop το 'SUPPLIER_TO'
df_representations_train.head()

,ACCT_NBR,SUCCESSOR,SUPPLIER,START_DATE
0,000A31FE38C6F648845941B642E287FE0200000043989C...,4,W,2017-04-04
1,000A31FE38C6F648845941B642E287FE020000004398BA...,1,Δ,1995-10-18
2,000A31FE38C6F648845941B642E287FE020000004398BA...,4,Δ,2019-11-06
3,000A31FE38C6F648845941B642E287FE020000004398BA...,5,Δ,2020-06-23
4,000A31FE38C6F648845941B642E287FE020000004398BD...,1,Δ,1995-10-15


In [ ]:
#same for test
df_representations_test.drop(['END_DATE'], axis = 1, inplace = True)

In [ ]:
df_requests_train.drop(['COMPL_REQUEST_STATUS', 'COMPLETION_DATE'], axis = 1, inplace = True)
df_requests_train.head()

,ACCT_NBR,SUCCESSOR,REQUEST_TYPE,REQUEST_DATE
0,000A31FE38C6F648845941B642E287FE020000000AA1CA...,7,recon,2019-02-12
1,000A31FE38C6F648845941B642E287FE020000000AA1D1...,3,NaN,2015-05-26
2,000A31FE38C6F648845941B642E287FE020000000AA1D1...,4,NaN,2017-02-06
3,000A31FE38C6F648845941B642E287FE020000000AA1D1...,5,discon,2018-06-28
4,000A31FE38C6F648845941B642E287FE020000000AA1D1...,5,reprChange,2018-09-27


In [ ]:
#test
df_requests_test.drop(['COMPL_REQUEST_STATUS', 'COMPLETION_DATE'], axis = 1, inplace = True)

In [ ]:
df_consumptions_train.drop(['BS_RATE'], axis = 1, inplace = True)
df_consumptions_train.head()

,ACCT_NBR,SUCCESSOR,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
0,000A31FE38C6F648845941B642E287FE02000000B7602D...,1,000A31FE38C6F648845941B642E287FE0200000032011D...,56,2019-08-26
1,000A31FE38C6F648845941B642E287FE020000003CC307...,1,000A31FE38C6F648845941B642E287FE020000002BFDE8...,2236,2022-03-10
2,000A31FE38C6F648845941B642E287FE02000000543AF1...,1,000A31FE38C6F648845941B642E287FE0200000055887E...,678,2021-04-20
3,000A31FE38C6F648845941B642E287FE02000000617012...,1,000A31FE38C6F648845941B642E287FE02000000310114...,156,2022-04-27
4,000A31FE38C6F648845941B642E287FE020000006CBF0C...,1,000A31FE38C6F648845941B642E287FE02000000B94B46...,502,2022-06-03


In [ ]:
df_consumptions_test.drop(['BS_RATE'], axis = 1, inplace = True)

#SQL

##Representations:  group by ACCT_NBR & SUCCESSOR   order by HM/NIA ΕΝΑΡΞΗΣ  θελουμε sum SUPPLIER και τελευταιο SUPPLIER 

SQL: 
SELECT ACCT_NBR, SUCCESSOR, 
    SUM(SUPPLIER) as TOTAL_SUPPLIER, 
    LAST_VALUE(SUPPLIER) OVER (PARTITION BY ACCT_NBR, SUCCESSOR ORDER BY START_DATE) as LAST_SUPPLIER
FROM REPRESENTATIONS
GROUP BY ACCT_NBR, SUCCESSOR
ORDER BY START_DATE


In [ ]:
# Calculate the last supplier by account and successor
last_supplier = df_representations_train.groupby(['ACCT_NBR', 'SUCCESSOR'])['SUPPLIER'].last().reset_index()
last_supplier.columns = ['ACCT_NBR', 'SUCCESSOR', 'LAST_SUPPLIER']
last_supplier.head()

,ACCT_NBR,SUCCESSOR,LAST_SUPPLIER
0,000A31FE38C6F648845941B642E287FE0200000000004A...,1,Δ
1,000A31FE38C6F648845941B642E287FE0200000000004A...,2,Δ
2,000A31FE38C6F648845941B642E287FE02000000000050...,1,Δ
3,000A31FE38C6F648845941B642E287FE0200000000005D...,1,Δ
4,000A31FE38C6F648845941B642E287FE0200000000007D...,1,Δ


In [ ]:
#test
last_supplier_test = df_representations_test.groupby(['ACCT_NBR', 'SUCCESSOR'])['SUPPLIER'].last().reset_index()
last_supplier_test.columns = ['ACCT_NBR', 'SUCCESSOR', 'LAST_SUPPLIER']
last_supplier_test.head()

,ACCT_NBR,SUCCESSOR,LAST_SUPPLIER
0,000A31FE38C6F648845941B642E287FE0200000000006D...,1,Δ
1,000A31FE38C6F648845941B642E287FE0200000000006D...,2,L
2,000A31FE38C6F648845941B642E287FE0200000000006D...,3,W
3,000A31FE38C6F648845941B642E287FE0200000000006E...,1,Δ
4,000A31FE38C6F648845941B642E287FE020000000001BF...,1,Δ


In [ ]:
# Calculate the total supplier 
total_supplier = df_representations_train.groupby(['ACCT_NBR', 'SUCCESSOR'])['SUPPLIER'].nunique().reset_index()
total_supplier.columns = ['ACCT_NBR', 'SUCCESSOR', 'TOTAL_SUPPLIER']
total_supplier.head()

,ACCT_NBR,SUCCESSOR,TOTAL_SUPPLIER
0,000A31FE38C6F648845941B642E287FE0200000000004A...,1,1
1,000A31FE38C6F648845941B642E287FE0200000000004A...,2,1
2,000A31FE38C6F648845941B642E287FE02000000000050...,1,1
3,000A31FE38C6F648845941B642E287FE0200000000005D...,1,1
4,000A31FE38C6F648845941B642E287FE0200000000007D...,1,1


In [ ]:
# test 
total_supplier_test = df_representations_test.groupby(['ACCT_NBR', 'SUCCESSOR'])['SUPPLIER'].nunique().reset_index()
total_supplier_test.columns = ['ACCT_NBR', 'SUCCESSOR', 'TOTAL_SUPPLIER']
total_supplier_test.head()

,ACCT_NBR,SUCCESSOR,TOTAL_SUPPLIER
0,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1
1,000A31FE38C6F648845941B642E287FE0200000000006D...,2,1
2,000A31FE38C6F648845941B642E287FE0200000000006D...,3,1
3,000A31FE38C6F648845941B642E287FE0200000000006E...,1,1
4,000A31FE38C6F648845941B642E287FE020000000001BF...,1,1


In [ ]:
#merge
result_rep = pd.merge(total_supplier, last_supplier, on=['ACCT_NBR', 'SUCCESSOR'])
result_rep = pd.merge(result_rep, df_representations_train, on=['ACCT_NBR', 'SUCCESSOR'])
result_rep.head()

,ACCT_NBR,SUCCESSOR,TOTAL_SUPPLIER,LAST_SUPPLIER,SUPPLIER,START_DATE
0,000A31FE38C6F648845941B642E287FE0200000000004A...,1,1,Δ,Δ,2000-04-07
1,000A31FE38C6F648845941B642E287FE0200000000004A...,2,1,Δ,Δ,2020-07-02
2,000A31FE38C6F648845941B642E287FE02000000000050...,1,1,Δ,Δ,2002-07-05
3,000A31FE38C6F648845941B642E287FE0200000000005D...,1,1,Δ,Δ,2001-12-27
4,000A31FE38C6F648845941B642E287FE0200000000007D...,1,1,Δ,Δ,2000-10-12


In [ ]:
#test
result_rep_test = pd.merge(total_supplier_test, last_supplier_test, on=['ACCT_NBR', 'SUCCESSOR'])
result_rep_test = pd.merge(result_rep_test, df_representations_test, on=['ACCT_NBR', 'SUCCESSOR'])
result_rep_test.head()

,ACCT_NBR,SUCCESSOR,TOTAL_SUPPLIER,LAST_SUPPLIER,SUPPLIER,START_DATE
0,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,Δ,Δ,2001-02-08
1,000A31FE38C6F648845941B642E287FE0200000000006D...,2,1,L,L,2021-03-08
2,000A31FE38C6F648845941B642E287FE0200000000006D...,3,1,W,W,2022-06-28
3,000A31FE38C6F648845941B642E287FE0200000000006E...,1,1,Δ,Δ,1995-10-22
4,000A31FE38C6F648845941B642E287FE020000000001BF...,1,1,Δ,Δ,1999-01-11


In [ ]:
result_rep = result_rep.sort_values(by='START_DATE',  ascending= True)
result_rep.head()

,ACCT_NBR,SUCCESSOR,TOTAL_SUPPLIER,LAST_SUPPLIER,SUPPLIER,START_DATE
500832,000A31FE38C6F648845941B642E287FE020000006838AE...,1,1,Δ,Δ,1199-11-22
547413,000A31FE38C6F648845941B642E287FE0200000071BF15...,1,1,Δ,Δ,1199-12-08
742733,000A31FE38C6F648845941B642E287FE020000009A4781...,1,1,Δ,Δ,1983-12-31
430457,000A31FE38C6F648845941B642E287FE020000005957B0...,1,1,Δ,Δ,1983-12-31
781795,000A31FE38C6F648845941B642E287FE02000000A271B6...,1,1,Δ,Δ,1983-12-31


In [ ]:
#test
result_rep_test = result_rep_test.sort_values(by='START_DATE',  ascending= True)
result_rep_test.head()

,ACCT_NBR,SUCCESSOR,TOTAL_SUPPLIER,LAST_SUPPLIER,SUPPLIER,START_DATE
57898,000A31FE38C6F648845941B642E287FE020000006D291A...,1,1,Δ,Δ,1983-12-31
117124,000A31FE38C6F648845941B642E287FE02000000DB9213...,1,1,Δ,Δ,1984-01-01
26637,000A31FE38C6F648845941B642E287FE0200000031D089...,1,1,Δ,Δ,1984-01-01
6587,000A31FE38C6F648845941B642E287FE020000000C2164...,1,1,Δ,Δ,1984-01-01
41236,000A31FE38C6F648845941B642E287FE020000004D2BCB...,1,1,Δ,Δ,1984-01-01


πετάω τις πρώτες 2 γραμμές απλό το train

In [ ]:
result_rep = result_rep.iloc[2:]
result_rep.head()

,ACCT_NBR,SUCCESSOR,TOTAL_SUPPLIER,LAST_SUPPLIER,SUPPLIER,START_DATE
742733,000A31FE38C6F648845941B642E287FE020000009A4781...,1,1,Δ,Δ,1983-12-31
430457,000A31FE38C6F648845941B642E287FE020000005957B0...,1,1,Δ,Δ,1983-12-31
781795,000A31FE38C6F648845941B642E287FE02000000A271B6...,1,1,Δ,Δ,1983-12-31
619043,000A31FE38C6F648845941B642E287FE02000000809CB0...,1,1,Δ,Δ,1983-12-31
1071009,000A31FE38C6F648845941B642E287FE02000000DEA5A4...,1,1,Δ,Δ,1983-12-31


###Δεν τα αποθηκεύω

In [ ]:
#$path = "/content/drive/MyDrive/HEDNO/result_rep.csv"
#with open(path, 'w', encoding = 'utf-8-sig') as f:
#  result_rep.to_csv(f)

In [ ]:
#path = "/content/drive/MyDrive/HEDNO/result_rep_test.csv"
#with open(path, 'w', encoding = 'utf-8-sig') as f:
#  result_rep_test.to_csv(f)

##Requests:  group by ACCT_NBR & SUCCESSOR order by REQUEST_DATE sum(REQUEST_TYPE) και τελευταιο REQUEST

SELECT ACCT_NBR, SUCCESSOR, 
    SUM(REQUEST_TYPE) as TOTAL_REQUEST_TYPE,
    LAST_VALUE(REQUEST_TYPE) OVER (PARTITION BY ACCT_NBR, SUCCESSOR ORDER BY REQUEST_DATE) as LAST_REQUEST_TYPE
FROM Requests
GROUP BY ACCT_NBR, SUCCESSOR
ORDER BY REQUEST_DATE

In [ ]:
# Calculate the total request type by account and successor
#value_counts για total_requwst_type
total_request_type = df_requests_train.groupby(['ACCT_NBR', 'SUCCESSOR'])['REQUEST_TYPE'].count().reset_index()
total_request_type.columns = ['ACCT_NBR', 'SUCCESSOR', 'TOTAL_REQUEST_TYPE']
total_request_type.head()

,ACCT_NBR,SUCCESSOR,TOTAL_REQUEST_TYPE
0,000A31FE38C6F648845941B642E287FE02000000000046...,3,2
1,000A31FE38C6F648845941B642E287FE02000000000046...,4,0
2,000A31FE38C6F648845941B642E287FE02000000000046...,5,1
3,000A31FE38C6F648845941B642E287FE0200000000007D...,1,1
4,000A31FE38C6F648845941B642E287FE02000000000080...,3,3


In [ ]:
# test
total_request_type_test = df_requests_test.groupby(['ACCT_NBR', 'SUCCESSOR'])['REQUEST_TYPE'].count().reset_index()
total_request_type_test.columns = ['ACCT_NBR', 'SUCCESSOR', 'TOTAL_REQUEST_TYPE']
total_request_type_test.head()

,ACCT_NBR,SUCCESSOR,TOTAL_REQUEST_TYPE
0,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1
1,000A31FE38C6F648845941B642E287FE0200000000006D...,2,1
2,000A31FE38C6F648845941B642E287FE020000000001BF...,1,7
3,000A31FE38C6F648845941B642E287FE02000000000536...,2,1
4,000A31FE38C6F648845941B642E287FE02000000000536...,3,6


In [ ]:
# Calculate the last request type by account and successor
last_request_type = df_requests_train.groupby(['ACCT_NBR', 'SUCCESSOR'])['REQUEST_TYPE'].last().reset_index()
last_request_type.columns = ['ACCT_NBR', 'SUCCESSOR', 'LAST_REQUEST_TYPE']
last_request_type.head()

,ACCT_NBR,SUCCESSOR,LAST_REQUEST_TYPE
0,000A31FE38C6F648845941B642E287FE02000000000046...,3,discon
1,000A31FE38C6F648845941B642E287FE02000000000046...,4,None
2,000A31FE38C6F648845941B642E287FE02000000000046...,5,discon
3,000A31FE38C6F648845941B642E287FE0200000000007D...,1,reprChange
4,000A31FE38C6F648845941B642E287FE02000000000080...,3,recon


In [ ]:
# test
last_request_type_test = df_requests_test.groupby(['ACCT_NBR', 'SUCCESSOR'])['REQUEST_TYPE'].last().reset_index()
last_request_type_test.columns = ['ACCT_NBR', 'SUCCESSOR', 'LAST_REQUEST_TYPE']
last_request_type_test.head()

,ACCT_NBR,SUCCESSOR,LAST_REQUEST_TYPE
0,000A31FE38C6F648845941B642E287FE0200000000006D...,1,reprChange
1,000A31FE38C6F648845941B642E287FE0200000000006D...,2,reprChange
2,000A31FE38C6F648845941B642E287FE020000000001BF...,1,discon
3,000A31FE38C6F648845941B642E287FE02000000000536...,2,reprChange
4,000A31FE38C6F648845941B642E287FE02000000000536...,3,discon


In [ ]:
#merge
result_req = pd.merge(total_request_type, last_request_type, on=['ACCT_NBR', 'SUCCESSOR'])
#result_req = pd.merge(result_req, df_requests_train, on=['ACCT_NBR', 'SUCCESSOR'])
result_req.head()

,ACCT_NBR,SUCCESSOR,TOTAL_REQUEST_TYPE,LAST_REQUEST_TYPE
0,000A31FE38C6F648845941B642E287FE02000000000046...,3,2,discon
1,000A31FE38C6F648845941B642E287FE02000000000046...,4,0,None
2,000A31FE38C6F648845941B642E287FE02000000000046...,5,1,discon
3,000A31FE38C6F648845941B642E287FE0200000000007D...,1,1,reprChange
4,000A31FE38C6F648845941B642E287FE02000000000080...,3,3,recon


In [ ]:
#test
result_req_test = pd.merge(total_request_type_test, last_request_type_test, on=['ACCT_NBR', 'SUCCESSOR'])
#result_req = pd.merge(result_req, df_requests_train, on=['ACCT_NBR', 'SUCCESSOR'])
result_req_test.head()

,ACCT_NBR,SUCCESSOR,TOTAL_REQUEST_TYPE,LAST_REQUEST_TYPE
0,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,reprChange
1,000A31FE38C6F648845941B642E287FE0200000000006D...,2,1,reprChange
2,000A31FE38C6F648845941B642E287FE020000000001BF...,1,7,discon
3,000A31FE38C6F648845941B642E287FE02000000000536...,2,1,reprChange
4,000A31FE38C6F648845941B642E287FE02000000000536...,3,6,discon


###Δεν αποθηκεύω

In [ ]:
#path = "/content/drive/MyDrive/HEDNO/result_req.csv"
#with open(path, 'w', encoding = 'utf-8-sig') as f:
#  result_req.to_csv(f)

In [ ]:
#path = "/content/drive/MyDrive/HEDNO/result_req_test.csv"
#with open(path, 'w', encoding = 'utf-8-sig') as f:
#  result_req_test.to_csv(f)

##Consumptions group by ACCT_NBR & SUCCESSOR   order by MEASUREMENT_DATE  θελουμε  τελευταιa 50  CSS_MS_HS_USE (καταναλωση) και SUM αριθμων μετρηση 



SELECT ACCT_NBR, SUCCESSOR, SUM(MS_METER_NBR) as TOTAL_MS_METER_NBR
FROM Consumptions
GROUP BY ACCT_NBR, SUCCESSOR
ORDER BY MEASUREMENT_DATE CSS_MS_HS_USE
ORDER BY MEASUREMENT_DATE

In [ ]:
#Sort by MEASUREMENT_DATE
#df_consumptions_train = df_consumptions_train.sort_values(by='MEASUREMENT_DATE', ascending=False)

In [ ]:
#Group by ACCT_NBR, SUCCESSOR, and CSS_MS_HS_USE
#grouped = df_consumptions_train.groupby(['ACCT_NBR', 'SUCCESSOR', 'CSS_MS_HS_USE'])

In [ ]:
#new df with last 50 rows for each group
#last_50 = grouped.apply(lambda x: x.head(50)).reset_index(drop=True)
#αργεί πάααρα πολύ να φορτώσει

In [ ]:
#total MS_METER_NBR by ACCT_NBR and SUCCESSOR
total_ms_meter_nbr = df_consumptions_train.groupby(['ACCT_NBR', 'SUCCESSOR'])['MS_METER_NBR'].nunique().reset_index()
total_ms_meter_nbr.columns = ['ACCT_NBR', 'SUCCESSOR', 'TOTAL_MS_METER_NBR']
total_ms_meter_nbr.head()

,ACCT_NBR,SUCCESSOR,TOTAL_MS_METER_NBR
0,000A31FE38C6F648845941B642E287FE0200000000004A...,1,1
1,000A31FE38C6F648845941B642E287FE0200000000004A...,2,2
2,000A31FE38C6F648845941B642E287FE02000000000050...,1,1
3,000A31FE38C6F648845941B642E287FE0200000000005D...,1,1
4,000A31FE38C6F648845941B642E287FE0200000000007D...,1,1


In [ ]:
#test
total_ms_meter_nbr_test = df_consumptions_test.groupby(['ACCT_NBR', 'SUCCESSOR'])['MS_METER_NBR'].nunique().reset_index()
total_ms_meter_nbr_test.columns = ['ACCT_NBR', 'SUCCESSOR', 'TOTAL_MS_METER_NBR']
total_ms_meter_nbr_test.head()

,ACCT_NBR,SUCCESSOR,TOTAL_MS_METER_NBR
0,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1
1,000A31FE38C6F648845941B642E287FE0200000000006D...,2,1
2,000A31FE38C6F648845941B642E287FE0200000000006D...,3,1
3,000A31FE38C6F648845941B642E287FE020000000001BF...,1,1
4,000A31FE38C6F648845941B642E287FE020000000001BF...,2,1


In [ ]:
total_ms_meter_nbr=pd.merge(total_ms_meter_nbr,df_consumptions_train, on=['ACCT_NBR','SUCCESSOR'], how='left')
total_ms_meter_nbr.head()

,ACCT_NBR,SUCCESSOR,TOTAL_MS_METER_NBR,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
0,000A31FE38C6F648845941B642E287FE0200000000004A...,1,1,000A31FE38C6F648845941B642E287FE020000007FDE53...,33,2018-11-30
1,000A31FE38C6F648845941B642E287FE0200000000004A...,1,1,000A31FE38C6F648845941B642E287FE020000007FDE53...,12,2018-03-30
2,000A31FE38C6F648845941B642E287FE0200000000004A...,1,1,000A31FE38C6F648845941B642E287FE020000007FDE53...,77,2019-12-02
3,000A31FE38C6F648845941B642E287FE0200000000004A...,1,1,000A31FE38C6F648845941B642E287FE020000007FDE53...,135,2020-07-02
4,000A31FE38C6F648845941B642E287FE0200000000004A...,2,2,000A31FE38C6F648845941B642E287FE020000007FDE53...,67,2020-09-17


In [ ]:
#test
total_ms_meter_nbr_test = pd.merge(total_ms_meter_nbr_test,df_consumptions_test, on=['ACCT_NBR','SUCCESSOR'], how='left')
total_ms_meter_nbr_test.head()

,ACCT_NBR,SUCCESSOR,TOTAL_MS_METER_NBR,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
0,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,000A31FE38C6F648845941B642E287FE02000000B2DCB1...,241,2019-10-02
1,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,000A31FE38C6F648845941B642E287FE02000000B2DCB1...,926,2019-02-01
2,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,000A31FE38C6F648845941B642E287FE02000000B2DCB1...,557,2020-10-05
3,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,000A31FE38C6F648845941B642E287FE02000000B2DCB1...,44,2021-03-08
4,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,000A31FE38C6F648845941B642E287FE02000000B2DCB1...,68,2021-02-05


In [ ]:
total_ms_meter_nbr.drop(['MS_METER_NBR', 'CSS_MS_HS_USE'], axis=1)

,ACCT_NBR,SUCCESSOR,TOTAL_MS_METER_NBR,MEASUREMENT_DATE
0,000A31FE38C6F648845941B642E287FE0200000000004A...,1,1,2018-11-30
1,000A31FE38C6F648845941B642E287FE0200000000004A...,1,1,2018-03-30
2,000A31FE38C6F648845941B642E287FE0200000000004A...,1,1,2019-12-02
3,000A31FE38C6F648845941B642E287FE0200000000004A...,1,1,2020-07-02
4,000A31FE38C6F648845941B642E287FE0200000000004A...,2,2,2020-09-17
...,...,...,...,...
9201389,000A31FE38C6F648845941B642E287FE02000000FFFFD8...,3,2,2021-10-04
9201390,000A31FE38C6F648845941B642E287FE02000000FFFFEB...,1,2,2022-05-23
9201391,000A31FE38C6F648845941B642E287FE02000000FFFFEB...,1,2,2022-06-30
9201392,000A31FE38C6F648845941B642E287FE02000000FFFFEB...,1,2,2022-05-23


In [ ]:
total_ms_meter_nbr_test.drop(['MS_METER_NBR', 'CSS_MS_HS_USE'], axis=1)

,ACCT_NBR,SUCCESSOR,TOTAL_MS_METER_NBR,MEASUREMENT_DATE
0,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,2019-10-02
1,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,2019-02-01
2,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,2020-10-05
3,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,2021-03-08
4,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,2021-02-05
...,...,...,...,...
718163,000A31FE38C6F648845941B642E287FE02000000FFFD63...,2,1,2021-12-20
718164,000A31FE38C6F648845941B642E287FE02000000FFFD63...,2,1,2021-10-11
718165,000A31FE38C6F648845941B642E287FE02000000FFFD63...,2,1,2018-04-17
718166,000A31FE38C6F648845941B642E287FE02000000FFFD63...,2,1,2020-10-12


In [ ]:
# Sort by MEASUREMENT_DATE
result_con = total_ms_meter_nbr.sort_values(by='MEASUREMENT_DATE' , ascending= True)
result_con.head()

,ACCT_NBR,SUCCESSOR,TOTAL_MS_METER_NBR,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
8204173,000A31FE38C6F648845941B642E287FE02000000E4558A...,5,2,000A31FE38C6F648845941B642E287FE02000000138B81...,1760,2018-01-01
8928776,000A31FE38C6F648845941B642E287FE02000000F88245...,3,8,000A31FE38C6F648845941B642E287FE0200000010A632...,0,2018-01-01
3341227,000A31FE38C6F648845941B642E287FE020000005CF571...,2,2,000A31FE38C6F648845941B642E287FE02000000308CDE...,2480,2018-01-01
3341224,000A31FE38C6F648845941B642E287FE020000005CF571...,2,2,000A31FE38C6F648845941B642E287FE020000007D47A9...,640,2018-01-01
891559,000A31FE38C6F648845941B642E287FE020000001900AE...,3,53,000A31FE38C6F648845941B642E287FE020000007FB31A...,7652,2018-01-01


In [ ]:
# test
result_con_test = total_ms_meter_nbr_test.sort_values(by='MEASUREMENT_DATE' , ascending= True)
result_con_test.head()

,ACCT_NBR,SUCCESSOR,TOTAL_MS_METER_NBR,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
397129,000A31FE38C6F648845941B642E287FE020000008E7527...,1,1,000A31FE38C6F648845941B642E287FE02000000C99869...,840,2018-01-01
572567,000A31FE38C6F648845941B642E287FE02000000CC363C...,5,2,000A31FE38C6F648845941B642E287FE02000000800DDA...,7680,2018-01-01
55146,000A31FE38C6F648845941B642E287FE0200000014066B...,2,2,000A31FE38C6F648845941B642E287FE020000007CB79A...,5000,2018-01-01
56080,000A31FE38C6F648845941B642E287FE02000000143979...,2,2,000A31FE38C6F648845941B642E287FE02000000A70865...,10520,2018-01-01
635957,000A31FE38C6F648845941B642E287FE02000000E29B70...,3,2,000A31FE38C6F648845941B642E287FE02000000FB7066...,520,2018-01-01


###Δεν αποθηκεύω

In [ ]:
#path = "/content/drive/MyDrive/HEDNO/result_con.csv"
#with open(path, 'w', encoding = 'utf-8-sig') as f:
#  result_con.to_csv(f)

In [ ]:
#path = "/content/drive/MyDrive/HEDNO/result_con_test.csv"
#with open(path, 'w', encoding = 'utf-8-sig') as f:
#  result_con_test.to_csv(f)

In [ ]:
#uniqueValuesCount = (df_records_train[['ACCT_NBR','SUCCESSOR']]).nunique()
#print(uniqueValuesCount )

##Δημιουργώ το dataframe

In [ ]:
new_df=pd.merge(result_req,result_rep, how='left',on=['ACCT_NBR','SUCCESSOR'])

In [ ]:
#test
new_df_test=pd.merge(result_req_test,result_rep_test, how='left',on=['ACCT_NBR','SUCCESSOR'])

In [ ]:
new_df=pd.merge(new_df,result_con, how='left',on=['ACCT_NBR','SUCCESSOR'] )

In [ ]:
#test
new_df_test=pd.merge(new_df_test,result_con_test, how='left',on=['ACCT_NBR','SUCCESSOR'] )

In [ ]:
new_df.head()

,ACCT_NBR,SUCCESSOR,TOTAL_REQUEST_TYPE,LAST_REQUEST_TYPE,TOTAL_SUPPLIER,LAST_SUPPLIER,SUPPLIER,START_DATE,TOTAL_MS_METER_NBR,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
0,000A31FE38C6F648845941B642E287FE02000000000046...,3,2,discon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,000A31FE38C6F648845941B642E287FE02000000000046...,4,0,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,000A31FE38C6F648845941B642E287FE02000000000046...,5,1,discon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,000A31FE38C6F648845941B642E287FE0200000000007D...,1,1,reprChange,1.0,Δ,Δ,2000-10-12,1.0,000A31FE38C6F648845941B642E287FE020000003A5267...,1714.0,2018-02-06
4,000A31FE38C6F648845941B642E287FE0200000000007D...,1,1,reprChange,1.0,Δ,Δ,2000-10-12,1.0,000A31FE38C6F648845941B642E287FE020000003A5267...,1430.0,2018-06-07


In [ ]:
new_df_test.head()

,ACCT_NBR,SUCCESSOR,TOTAL_REQUEST_TYPE,LAST_REQUEST_TYPE,TOTAL_SUPPLIER,LAST_SUPPLIER,SUPPLIER,START_DATE,TOTAL_MS_METER_NBR,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
0,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,reprChange,1.0,Δ,Δ,2001-02-08,1.0,000A31FE38C6F648845941B642E287FE02000000B2DCB1...,297.0,2018-02-02
1,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,reprChange,1.0,Δ,Δ,2001-02-08,1.0,000A31FE38C6F648845941B642E287FE02000000B2DCB1...,926.0,2019-02-01
2,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,reprChange,1.0,Δ,Δ,2001-02-08,1.0,000A31FE38C6F648845941B642E287FE02000000B2DCB1...,351.0,2019-06-04
3,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,reprChange,1.0,Δ,Δ,2001-02-08,1.0,000A31FE38C6F648845941B642E287FE02000000B2DCB1...,241.0,2019-10-02
4,000A31FE38C6F648845941B642E287FE0200000000006D...,1,1,reprChange,1.0,Δ,Δ,2001-02-08,1.0,000A31FE38C6F648845941B642E287FE02000000B2DCB1...,348.0,2020-02-05


#Reset Indexes ΑΧΡΗΣΤΟ ΤΕΛΙΚΑ

In [ ]:
result_con_test.reset_index(drop=True, inplace=True)

In [ ]:
result_con.reset_index(drop=True, inplace=True)

In [ ]:
result_req.reset_index(drop=True, inplace=True)

In [ ]:
result_req_test.reset_index(drop=True, inplace=True)

In [ ]:
result_rep.reset_index(drop=True, inplace=True)

In [ ]:
result_rep_test.reset_index(drop=True, inplace=True)

#Add column to power thefts with 1

In [ ]:
df_power_thefts['target'] = 1

In [ ]:
df_power_thefts.head()

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY,target
0,000A31FE38C6F648845941B642E287FE02000000004D63...,3,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2018-03-01,2018-03-01,NaN,1283.0,1
1,000A31FE38C6F648845941B642E287FE0200000000801E...,8,Μέσω Καταμέτρησης (ΔΑΚ),ΑΚΡΟΔΕΚΤΕΣ,2018-08-16,2018-11-01,2018-11-01,640.0,1
2,000A31FE38C6F648845941B642E287FE0200000000B03B...,2,Μέσω Λοιπών Εργασιών & Συντήρησης,ΜΗΧΑΝΙΣΜΟΣ ΜΕΤΡΗΣΗΣ,2018-12-05,2018-12-18,NaN,4665.0,1
3,000A31FE38C6F648845941B642E287FE0200000000BCD9...,2,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΚΡΟΔΕΚΤΕΣ,2017-01-19,2017-01-30,NaN,NaN,1
4,000A31FE38C6F648845941B642E287FE02000000017AA3...,5,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-05-29,2019-05-29,NaN,1049.0,1


In [ ]:
col_names = list(df_power_thefts.columns)
print(col_names)

['ACCT_NBR', 'SUCCESSOR', 'ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ', 'ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ', 'HMANAF', 'INITIAL_DETECTION_DATE', 'DETECTION_DATE', 'NON_REGISTERED_ENERGY', 'target']


In [ ]:
df_power_thefts.drop(['DETECTION_DATE','HMANAF'], axis = 1, inplace = True)

#Join datasets except power_thefts with inner join

In [ ]:
fin_info = pd.merge(result_rep, result_req , on=['ACCT_NBR', 'SUCCESSOR'], how='inner')

In [ ]:
fin_info = pd.merge(fin_info, result_con , on=['ACCT_NBR', 'SUCCESSOR'], how='inner')

test

In [ ]:
fin_info_test = pd.merge(result_rep_test , result_req_test , on=['ACCT_NBR', 'SUCCESSOR'], how='inner')

In [ ]:
fin_info_test = pd.merge(fin_info_test, result_con_test , on=['ACCT_NBR', 'SUCCESSOR'], how='inner')

#Left Join fin_info with power_thefts and to target column where value=Nan replace with 0

In [ ]:
fin_info_1 = pd.merge(fin_info, df_power_thefts , on=['ACCT_NBR', 'SUCCESSOR'], how='left')

In [ ]:
fin_info_1['target'] = fin_info_1['target'].fillna(0)

In [ ]:
fin_info_1["target"] = fin_info_1["target"].astype(int)

In [ ]:
fin_info_1.head()

,ACCT_NBR,SUCCESSOR,TOTAL_SUPPLIER,LAST_SUPPLIER,SUPPLIER,START_DATE,TOTAL_REQUEST_TYPE,LAST_REQUEST_TYPE,TOTAL_MS_METER_NBR,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,INITIAL_DETECTION_DATE,NON_REGISTERED_ENERGY,target
0,000A31FE38C6F648845941B642E287FE020000009A4781...,1,1,Δ,Δ,1983-12-31,4,discon,2,000A31FE38C6F648845941B642E287FE02000000BC2966...,0,2018-11-14,NaN,NaN,NaN,NaN,0
1,000A31FE38C6F648845941B642E287FE020000009A4781...,1,1,Δ,Δ,1983-12-31,4,discon,2,000A31FE38C6F648845941B642E287FE0200000046FEBA...,6154,2018-11-14,NaN,NaN,NaN,NaN,0
2,000A31FE38C6F648845941B642E287FE020000005957B0...,1,1,Δ,Δ,1983-12-31,10,discon,2,000A31FE38C6F648845941B642E287FE02000000E84A6D...,0,2018-11-01,NaN,NaN,NaN,NaN,0
3,000A31FE38C6F648845941B642E287FE020000005957B0...,1,1,Δ,Δ,1983-12-31,10,discon,2,000A31FE38C6F648845941B642E287FE020000005CB361...,0,2018-11-01,NaN,NaN,NaN,NaN,0
4,000A31FE38C6F648845941B642E287FE02000000A271B6...,1,1,Δ,Δ,1983-12-31,18,recon,1,000A31FE38C6F648845941B642E287FE0200000010BA50...,1239,2018-02-26,NaN,NaN,NaN,NaN,0


In [ ]:
#fin_info_1.info()

#Στατιστική ανάλυση στα δεδομένα του τελικού dataframe - Not to run

##Ποσοστό εντοπισμένων ρευματοκλοπών στο δείγμα - σχεδον 10%

In [ ]:
#sns.histplot(data=fin_info_1, x='target', bins=30)

##Missing Values

In [ ]:
#total = fin_info_1.isnull().sum().sort_values(ascending=False)
#percent = (fin_info_1.isnull().sum()/fin_info_1.isnull().count()).sort_values(ascending=False)
#missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
#missing_data.head(20)

Κάνω drop τα ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ, ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ, NON_REGISTERED_ENERGY, INITIAL_DETECTION_DATE καθώς δεν τα χρειαστώ και στην τελική ανάλυση και έχουν πολλά missing data

In [ ]:
fin_info_1.drop(['ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ','ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ','NON_REGISTERED_ENERGY','INITIAL_DETECTION_DATE'], axis = 1, inplace = True)

In [ ]:
#fin_info_test.drop(['ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ','ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ','NON_REGISTERED_ENERGY','INITIAL_DETECTION_DATE'], axis = 1, inplace = True)

στο fin_info_test δεν τα έχω για να τα κάνω drop

##Handle missing data

In [ ]:
fin_info_1 = fin_info_1.apply(lambda x:x.fillna(x.value_counts().index[0]))
fin_info_1.isnull().sum().max()

0

In [ ]:
#test
fin_info_test = fin_info_test.apply(lambda x:x.fillna(x.value_counts().index[0]))
fin_info_test.isnull().sum().max()

0

In [ ]:
fin_info_1.head()

,ACCT_NBR,SUCCESSOR,TOTAL_SUPPLIER,LAST_SUPPLIER,SUPPLIER,START_DATE,TOTAL_REQUEST_TYPE,LAST_REQUEST_TYPE,TOTAL_MS_METER_NBR,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,INITIAL_DETECTION_DATE,NON_REGISTERED_ENERGY,target
0,000A31FE38C6F648845941B642E287FE020000009A4781...,1,1,Δ,Δ,1983-12-31,4,discon,2,000A31FE38C6F648845941B642E287FE02000000BC2966...,0,2018-11-14,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΚΡΟΔΕΚΤΕΣ,2019-10-14,0.0,0
1,000A31FE38C6F648845941B642E287FE020000009A4781...,1,1,Δ,Δ,1983-12-31,4,discon,2,000A31FE38C6F648845941B642E287FE0200000046FEBA...,6154,2018-11-14,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΚΡΟΔΕΚΤΕΣ,2019-10-14,0.0,0
2,000A31FE38C6F648845941B642E287FE020000005957B0...,1,1,Δ,Δ,1983-12-31,10,discon,2,000A31FE38C6F648845941B642E287FE02000000E84A6D...,0,2018-11-01,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΚΡΟΔΕΚΤΕΣ,2019-10-14,0.0,0
3,000A31FE38C6F648845941B642E287FE020000005957B0...,1,1,Δ,Δ,1983-12-31,10,discon,2,000A31FE38C6F648845941B642E287FE020000005CB361...,0,2018-11-01,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΚΡΟΔΕΚΤΕΣ,2019-10-14,0.0,0
4,000A31FE38C6F648845941B642E287FE02000000A271B6...,1,1,Δ,Δ,1983-12-31,18,recon,1,000A31FE38C6F648845941B642E287FE0200000010BA50...,1239,2018-02-26,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΚΡΟΔΕΚΤΕΣ,2019-10-14,0.0,0


In [ ]:
fin_info_test.head()

,ACCT_NBR,SUCCESSOR,TOTAL_SUPPLIER,LAST_SUPPLIER,SUPPLIER,START_DATE,TOTAL_REQUEST_TYPE,LAST_REQUEST_TYPE,TOTAL_MS_METER_NBR,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
0,000A31FE38C6F648845941B642E287FE0200000031D089...,1,1,Δ,Δ,1984-01-01,3,reprPause,2,000A31FE38C6F648845941B642E287FE02000000CBFDDA...,1352,2019-06-19
1,000A31FE38C6F648845941B642E287FE0200000031D089...,1,1,Δ,Δ,1984-01-01,3,reprPause,2,000A31FE38C6F648845941B642E287FE02000000ABD057...,25284,2019-06-19
2,000A31FE38C6F648845941B642E287FE0200000031D089...,1,1,Δ,Δ,1984-01-01,3,reprPause,2,000A31FE38C6F648845941B642E287FE02000000CBFDDA...,0,2019-11-15
3,000A31FE38C6F648845941B642E287FE0200000031D089...,1,1,Δ,Δ,1984-01-01,3,reprPause,2,000A31FE38C6F648845941B642E287FE02000000ABD057...,0,2019-11-15
4,000A31FE38C6F648845941B642E287FE020000004D2BCB...,1,1,Δ,Δ,1984-01-01,3,recon,1,000A31FE38C6F648845941B642E287FE02000000E5083A...,1302,2018-11-28


In [ ]:
fin_info_1.drop(['MS_METER_NBR','MEASUREMENT_DATE'], axis=1, inplace=True)

In [ ]:
fin_info_test.drop(['MS_METER_NBR','MEASUREMENT_DATE'], axis=1, inplace=True)

#Set multicolumn index

In [ ]:
#fin_info_1=fin_info_1.set_index(['ACCT_NBR', 'SUCCESSOR'])

In [ ]:
#fin_info_1.head()

In [ ]:
#test
#fin_info_test=fin_info_test.set_index(['ACCT_NBR', 'SUCCESSOR'])

#ΑΠΟΘΗΚΕΥΩ ΜΟΝΟ ΤΑ ΔΥΟ ΤΕΛΙΚΑ ΑΡΧΕΙΑ ΣΤΟ DRIVE ΚΑΙ ΘΑ ΤΑ ΔΟΥΛΕΨΟΥΜΕ ΣΕ ΑΛΛΟ COLAB

In [ ]:
path = "/content/drive/MyDrive/HEDNO/result_train.csv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  fin_info_1.to_csv(f)

In [ ]:
path = "/content/drive/MyDrive/HEDNO/result_test.csv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  fin_info_test.to_csv(f)